### Ноутбук подготовлен по материалам Артема Ерохина (X5 Group)

In [1]:
import numpy as np
from scipy.stats import chi2_contingency
from tqdm.notebook import tqdm

In [2]:
# На прошедшей неделе в рекламной сети параллельно размещались два баннера. 
# Оба баннера были показаны один миллион раз. Первый получил 10 000 кликов и 500 установок, а второй — 10 500 кликов и 440 установок. 
# Маркетолог просит у вас совета: какой баннер оставить, а какой отключить.
# Что вы ему ответите? Обоснуйте свой ответ.

In [3]:
# таблица с показами - были ли клик, или нет
table_click = [[10_000, 990_000], [10_500, 989_500]]
# таблица с установками из кликов - была ли установка из клика
table_click_install = [[500, 9_500], [440, 10_060]]
# таблица с установками из показов
table_install = [[500, 999_500], [440, 999_560]]

In [4]:
print("total click conversions 1st: {}, 2nd: {}".format(10_000 / 1_000_000, 10_500 / 1_000_000))
print("total install conversions 1st: {}, 2nd: {}".format(500 / 1_000_000, 440 / 1_000_000))
print("click to install conversions 1st: {}, 2nd: {}".format(500 / 10_000, 440 / 10_500))

total click conversions 1st: 0.01, 2nd: 0.0105
total install conversions 1st: 0.0005, 2nd: 0.00044
click to install conversions 1st: 0.05, 2nd: 0.0419047619047619


In [5]:
stat, p, dof, expected = chi2_contingency(table_click)
print(round(p,5))
alpha = 0.05
if p > alpha:
    print('Различий в эффекте нет')
else:
    print('Различия в эффекте есть')

0.00046
Различия в эффекте есть


In [6]:
stat, p, dof, expected = chi2_contingency(table_click_install)
print(round(p,5))
alpha = 0.05
if p > alpha:
    print('Различий в эффекте нет')
else:
    print('Различия в эффекте есть')

0.00621
Различия в эффекте есть


In [7]:
stat, p, dof, expected = chi2_contingency(table_install)
print(round(p,5))
alpha = 0.05
if p > alpha:
    print('Различий в эффекте нет')
else:
    print('Различия в эффекте есть')

0.05425
Различий в эффекте нет


In [8]:
# То есть у нас есть два значимых результата - по конверсиям из показов в клик и из кликов в установку, 
# которые будут значимы даже при поправке на множественность тестирования.
# Но при этом, у нас не является значимым тест конверсий из показов в установки (на уровне значимости 0.05). 
# Проверим бутстрепом последний результат. Дополнительно посмотрим конверсию в клики (вдруг нам все же важна именно эта конверсия).

In [9]:
def bootstrap(data, num_samples, sample_size, statistic, alpha):
    """Returns bootstrap estimate of 100.0*(1-alpha) CI for statistic."""
    sample_stat = []
    for _ in tqdm(range(num_samples)):
      sample_n = np.random.choice(data, size=sample_size)
      sample_stat.append(statistic(sample_n))
    conf_interval = np.percentile(sample_stat, [alpha / 2, 1 - alpha / 2])
    return conf_interval

In [10]:
a_group = np.hstack([np.zeros(999_500), np.ones(500)])
b_group = np.hstack([np.zeros(999_560), np.ones(440)])

In [11]:
arr1 = bootstrap(a_group, 1000, 1000000, np.mean, 0.05)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [12]:
arr2 = bootstrap(b_group, 1000, 1000000, np.mean, 0.05)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [13]:
print(arr1)
print(arr2)

[0.00043425 0.000447  ]
[0.0003615 0.000394 ]


In [14]:
a_group = np.hstack([np.zeros(990_000), np.ones(10_000)])
b_group = np.hstack([np.zeros(989_500), np.ones(10_500)])

In [15]:
arr3 = bootstrap(a_group, 1000, 1000000, np.mean, 0.05)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [16]:
arr4 = bootstrap(b_group, 1000, 1000000, np.mean, 0.05)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [17]:
print(arr3)
print(arr4)

[0.00960523 0.00978096]
[0.01016624 0.01026418]


In [ ]:
# Интервалы не пересекаются. В группе A получается лучшая конверсия из показов в установки. В группе B лучше конверсия из показов в клики.
# Получаем итог:
# 1. Eсли нам более важны итоговые установки - выбирается вариант с 10000 кликов и 500 установок из 1000000 показов.
# 2. Если нам важнее клики, то лучше выбрать группу с 10500 кликов и 440 установок из 1000000 показов.